In [1]:
from qtensor_ai.OpFactory import ParallelParametricGate, ParallelTorchFactory
from qtensor_ai import ParallelComposer
from qtensor_ai.Simulate import ParallelSimulator
from qtensor_ai.Backend import ParallelTorchBackend

import torch
import torch.nn as nn
import torch.multiprocessing as mp

import numpy as np
import itertools
from functools import partial
import time

2022-04-11 16:07:38,801- WARNING•	Tamaki solver is unavailable: No path /home/mliu6/.conda/envs/qtensor_ai/lib/python3.9/site-packages/qtensor_ai-0.1.0-py3.9.egg/qtensor_ai/qtensor/thirdparty/tamaki_treewidth. Either install tamaki in /home/mliu6/.conda/envs/qtensor_ai/lib/python3.9/site-packages/qtensor_ai-0.1.0-py3.9.egg/qtensor_ai/qtensor/thirdparty/tamaki_treewidth or add `tw-heuristic` to your $PATH


In [2]:
class RandU(ParallelParametricGate):
    name = 'Rand'
    _changes_qubits=(0, 1)

    @staticmethod
    def _gen_tensor(**parameters):
        return parameters['alpha']

    def gen_tensor(self, **parameters):
        if len(parameters) == 0:
            tensor = self._gen_tensor(**self._parameters)
        if self.is_inverse:
            tensor = torch.permute(tensor, [0,2,1,4,3]).conj()
        return tensor

    def __str__(self):
        return ("{}".format(self.name) +
                "({})".format(','.join(map(str, self._qubits)))
        )

ParallelTorchFactory.RandU = RandU


'''Compose the circuit to evaluate the trace of the target circuit'''
class TraceEvaluationComposer(ParallelComposer):
    
    def __init__(self, n_qubits, com):
        self.n_target_qubits = n_qubits
        self.n_qubits = n_qubits*2
        self.com = com
        super().__init__(n_qubits*2)

    def added_circuit(self):
        for target_qubit in range(self.n_target_qubits):
            control_qubit = target_qubit + self.n_target_qubits
            self.apply_gate(self.operators.H, control_qubit)
            self.apply_gate(self.operators.cX, control_qubit, target_qubit)

    '''Building circuit whose first amplitude is the expectation value of the measured circuit wrt to the cost_operator'''
    def updated_full_circuit(self, **parameters):



        '''This line of code where we used to have self.com.n_batch = self.n_batch is inconsistent with the HybridModule implementation'''
        


        circuit = self.com.updated_full_circuit(**parameters)


        self.n_batch = self.com.n_batch


        self.builder.reset()
        self.added_circuit()
        first_part = self.builder.circuit
        self.builder.inverse()
        second_part = self.builder.circuit
        self.builder.reset()
        result_circuit = first_part + circuit + second_part
        return result_circuit

    def name(self):
        return 'TraceEvaluation'


class LocalRandomUnitaryComposer(ParallelComposer):

    def __init__(self, n_qubits, n_layers):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        super().__init__(n_qubits)

    def _get_builder(self):
        return self._get_builder_class()(self.n_qubits)

    def updated_full_circuit(self, **parameters):
        unitaries = parameters['unitaries']
        self.n_batch = unitaries.shape[0]
        self.builder.reset()
        for qubit in range(self.n_qubits):
            self.apply_gate(self.operators.M, qubit)
        for layer in range(self.n_layers):
            qubit = np.random.randint(self.n_qubits-1)
            #print(unitaries[:, layer].shape)
            self.apply_gate(self.operators.RandU, qubit, qubit+1, alpha=unitaries[:, layer])
        return self.builder.circuit
    
    def name():
        return 'Local_Random'

In [7]:
def random_unitary_generator(n_batch, n_layers, n_qubits):
    module = nn.Linear(1,1)
    module.unitary = nn.Parameter(torch.rand(n_batch, n_layers, 2**n_qubits, 2**n_qubits, dtype=torch.cfloat))
    orthmod = nn.utils.parametrizations.orthogonal(module, name='unitary')
    results = orthmod.unitary.reshape(n_batch, n_layers, 2,2,2,2).detach()
    return results

def get_val(com, trace, sim, unitary):
    peo = None
    trace_circuit = trace.updated_full_circuit(unitaries=unitary)
    result = sim.simulate_batch(trace_circuit, peo=peo)
    return result.detach()

def get_vals(n_qubits, n_layers, num):

    com = LocalRandomUnitaryComposer(n_qubits, 2*n_layers)
    trace = TraceEvaluationComposer(n_qubits, com)
    sim = ParallelSimulator(backend=ParallelTorchBackend())
    get_val_map_fn = partial(get_val, com, trace, sim)
    pool = mp.Pool(num)
    #unitaries = [random_unitary_generator(1, 2*n_layers, 2)[0] for i in range(num)]
    unitaries = random_unitary_generator(num, 2*n_layers, 2).unsqueeze(1)
    result = pool.map(get_val_map_fn, unitaries)
    return torch.abs(torch.tensor(result))



In [8]:
n_qubits = 10
n_layers = n_qubits * 2
num = 30

com = LocalRandomUnitaryComposer(n_qubits, 2*n_layers)
trace = TraceEvaluationComposer(n_qubits, com)
sim = ParallelSimulator(backend=ParallelTorchBackend())

start = time.time()
print(get_val(com,trace,sim,random_unitary_generator(num, 2*n_layers, 2)[0].unsqueeze(0)))
stop = time.time()
print('Time taken for a single evaluation: ', stop - start)
start = time.time()
print(get_vals(n_qubits, n_layers, num))
stop = time.time()
print('Time taken for multiprocess evaluation: ', stop - start)

tensor([0.0004+0.0008j])
Time taken for a single evaluation:  0.05547332763671875
tensor([6.2445e-04, 2.7660e-03, 2.3442e-03, 1.8287e-03, 3.5030e-03, 9.5215e-04,
        6.4737e-04, 4.4468e-03, 1.2972e-03, 1.1672e-03, 1.3113e-03, 2.5305e-03,
        7.0406e-05, 3.4946e-04, 2.1406e-03, 1.0710e-03, 2.8717e-03, 1.6313e-03,
        3.2831e-04, 4.6847e-04, 2.0729e-03, 2.6349e-03, 4.4700e-04, 7.4173e-04,
        2.1781e-03, 2.5780e-03, 1.4669e-03, 1.7918e-03, 1.1295e-04, 9.3396e-04])
Time taken for multiprocess evaluation:  0.6076977252960205
